In [1]:
#from google.colab import drive
#import os

#drive.mount('/content/drive')

In [2]:
#cd drive/MyDrive

In [3]:
import pandas as pd
import numpy as np
from numpy import nan

In [ ]:
members = pd.read_csv('members.csv')
songs = pd.read_csv('songs.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
songs_extra = pd.read_csv('song_extra_info.csv')
sample = pd.read_csv('sample_submission.csv')

Первым делом кастанем все в нормальное время

In [ ]:
import datetime, time

def to_time(elem):
  elem = str(elem)
  t = datetime.datetime(int(elem[:4]), int(elem[4:6]), int(elem[6:]))
  return t.year * 365 + t.month * 30 + t.day

In [ ]:
members['registration_init_time'] = members['registration_init_time'].map(lambda elem : to_time(elem))
members['expiration_date'] = members['expiration_date'].map(lambda elem : to_time(elem))
members.head()

In [ ]:
def nan(df):
  for col in df.columns:
    print(df[col].isna().sum()) 
nan(songs)
songs.head()

In [ ]:
members['gender'] = members['gender'].fillna('Ungender')
members.head()

In [ ]:
songs = pd.read_csv('songs.csv')
songs['genre_ids'] = songs['genre_ids'].fillna(0)
songs['composer'] = songs['composer'].fillna('Unknown')
songs['lyricist'] = songs['lyricist'].fillna('Unknown')
songs['language'] = songs['language'].fillna(0)
def genre_dev(genre, ind):
    splitted = str(genre).split('|')
    if (len(splitted) <= ind):
        return 'na'
    else:
        return splitted[ind]
songs['g1'] = songs['genre_ids'].map(lambda g: genre_dev(g, 0))
songs['g2'] = songs['genre_ids'].map(lambda g: genre_dev(g, 1))
songs['g3'] = songs['genre_ids'].map(lambda g: genre_dev(g, 2))
songs = songs.drop(['genre_ids'], axis=1)


In [ ]:
train.head()
merged_train = train.merge(members, on='msno').merge(songs, on='song_id')
merged_train.dropna(inplace=True)
merged_train.reset_index(drop=True, inplace=True)
Y = merged_train['target']
X = merged_train.drop(['target'], axis=1)

In [ ]:
#from category_encoders import TargetEncoder
#X_encoded = X.copy()
#for col in ['msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 'city', 'bd', 'gender', 'registered_via', 'g1', 'g2', 'g3', 'artist_name', 'composer', 'lyricist', 'language']:
#  encoder = TargetEncoder()
#  X_encoded[col] = encoder.fit_transform(X_encoded[col].astype('category'), Y)
#X_encoded.head()
X

In [ ]:
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import cross_val_score, cross_validate

paramparam = ['msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 'city', 'bd', 'gender', 'registered_via', 'artist_name', 'composer', 'lyricist','g1', 'g2', 'g3']

cv_dataset = Pool(data=X,
                  label=Y,
                  cat_features = paramparam)

model = CatBoostRegressor(task_type='CPU', loss_function='RMSE', iterations=100)
model.fit(cv_dataset)
#model = CatBoostClassifier(iterations=1000, learning_rate=1e-5, depth=4, cat_features = ["msno", "song_id", "source_system_tab", "source_screen_name", "source_type", "gender"])
#model.fit(X, Y)

params = {"iterations": 100,
          "depth": 2,
          "loss_function": "RMSE",
          "verbose": False,
          "custom_metric": 'AUC'}


scores = cv(cv_dataset,
            params,
            fold_count=4,
            shuffle=True, 
            plot=True)

scores

In [ ]:
scores.iloc[-1]['test-AUC-mean']

In [ ]:
X